# Расчет логарифмических доходностей (Log Returns)

Базовый скрипт для расчета log returns на данных MOEX.

**Формула:** `log_return = ln(P_t / P_{t-1})`

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

Hello world


In [1]:
# Определение пути к данным
current_dir = Path.cwd()
DATA_DIR = current_dir / "data" / "MOEX_DATA"

# Поиск данных в альтернативных местах
if not DATA_DIR.exists():
    alt_path = current_dir.parent / "ML" / "data" / "MOEX_DATA"
    if alt_path.exists():
        DATA_DIR = alt_path
    else:
        for parent in [current_dir, current_dir.parent]:
            test_path = parent / "data" / "MOEX_DATA"
            if test_path.exists():
                DATA_DIR = test_path
                break

print(f"Путь к данным: {DATA_DIR}")
print(f"Найдено: {DATA_DIR.exists()}")

NameError: name 'Path' is not defined

In [ ]:
def calculate_log_returns(prices):
    """Вычисляет логарифмические доходности: log(P_t / P_{t-1})"""
    if not isinstance(prices, pd.Series):
        prices = pd.Series(prices)
    log_prices = np.log(prices)
    return log_prices - log_prices.shift(1)


def load_ticker_data(ticker, data_dir=DATA_DIR, timeframe='1D'):
    """Загружает данные по тикеру из CSV"""
    ticker_dir = data_dir / ticker / timeframe
    if not ticker_dir.exists():
        raise FileNotFoundError(f"Папка {ticker_dir} не найдена")
    
    csv_files = list(ticker_dir.glob("*.csv"))
    if not csv_files:
        raise FileNotFoundError(f"CSV файл не найден в {ticker_dir}")
    
    df = pd.read_csv(csv_files[0])
    
    # Преобразование дат
    if 'end' in df.columns:
        df['date'] = pd.to_datetime(df['end'])
    elif 'begin' in df.columns:
        df['date'] = pd.to_datetime(df['begin'])
    
    df = df.sort_values('date').reset_index(drop=True)
    return df

print("✅ Функции загружены")

Empty DataFrame
Columns: []
Index: []


In [ ]:
# Тестирование на одном тикере
ticker = 'SBER'
df = load_ticker_data(ticker, timeframe='1D')
df['log_return'] = calculate_log_returns(df['close'])

print(f"\nТикер: {ticker}")
print(f"Период: {df['date'].min().date()} → {df['date'].max().date()}")
print(f"Записей: {len(df)}")
print(f"\nСтатистика log returns:")
print(df['log_return'].describe())


In [ ]:
# Тестирование на нескольких тикерах
test_tickers = ['SBER', 'GAZP', 'LKOH', 'MTSS']
results = {}

for ticker in test_tickers:
    try:
        df = load_ticker_data(ticker, timeframe='1D')
        df['log_return'] = calculate_log_returns(df['close'])
        
        results[ticker] = {
            'data': df,
            'mean': df['log_return'].mean(),
            'std': df['log_return'].std(),
            'min': df['log_return'].min(),
            'max': df['log_return'].max(),
        }
        print(f"✅ {ticker}: {len(df)} записей")
    except Exception as e:
        print(f"❌ {ticker}: {e}")

# Сводная таблица
if results:
    summary = pd.DataFrame({
        'Тикер': list(results.keys()),
        'Записей': [len(results[k]['data']) for k in results.keys()],
        'Среднее': [results[k]['mean'] for k in results.keys()],
        'Ст.откл.': [results[k]['std'] for k in results.keys()],
    })
    print(f"\n{summary.to_string(index=False)}")


In [ ]:
# Визуализация: используйте функции из plots.ipynb
# %run plots.ipynb  # Запустите эту команду, чтобы загрузить функции визуализации

# Примеры использования:
# plot_price_and_returns(results['SBER']['data'], ticker='SBER')
# plot_multiple_returns(results, tickers=['SBER', 'GAZP', 'LKOH'])
# plot_returns_correlation(results)
# plot_boxplot_comparison(results)
